# Sistema de Modelado y Análisis de Supervivencia (médica)

En este notebook se detalla un sistema o secuencia de pasos a seguir que se pueden aplicar a cualquier modelo de Machine Learning que involucren problemas de tipo ***supervivencia***.

## 1. Dataset
* Cada fila representa a un paciente.
* El objetivo es predecir el riesgo (**risk**) de que un paciente muera
    * Esto se determina en la columna **event**.
    * La columna time indica que tenga el evento en ese tiempo dado.
       * Ejemplo: Que una persona tenga cancer (***event = True***) en un tiempo dado (***time***)
---
En los análisis de supervivencia tenemos:
* Los targets:
    * **event**: True/False (muere/no muere).
    * **time**: El tiempo cuando el evento ocurre.
* Explanatory variables:
    * El resto de variables (age, prior_therapy, etc)

Por lo cual la formula de la ecuación matemática del modelo será:
$$
risk = (w_0) + (w_1) \cdot age + (w_2) \cdot prior\_therapy
$$

Con los modelos de Machine Learning, se busca encontrar los mejores valores (optimización) para los pesos ***w1*** y ***w2*** de la ecuación anterior, para finalmente calcular el **risk** asociado.

In [62]:
import pandas as pd

df_patients = pd.read_excel("../data/data_lung_cancer_smote.xlsx")
list_columns_categorical = df_patients.select_dtypes(include="object").columns
df_patients[list_columns_categorical] = df_patients[list_columns_categorical].astype("category")        # Transformación Object a Category. Paso importante para que OneHotEncoder() reconozca las variables categóricas y las transforme.
df_patients

,event,time,age,karnofsky_score,months_from_diagnosis,prior_therapy,treatment,celltype
0,True,2.373626,69.000000,60.000000,7.000000,No,Standard,Squamous
1,True,7.516484,38.000000,60.000000,3.000000,No,Standard,Squamous
2,True,4.153846,63.000000,60.000000,9.000000,Yes,Standard,Squamous
3,True,3.890110,65.000000,70.000000,11.000000,Yes,Standard,Squamous
4,True,0.329670,49.000000,20.000000,5.000000,No,Standard,Squamous
...,...,...,...,...,...,...,...,...
238,False,3.142881,65.810046,64.640822,4.762009,No,Standard,Smallcell
239,False,3.380047,36.495508,70.684273,21.551683,Yes,Test,Smallcell
240,False,3.082424,65.029553,81.087920,4.852974,No,Standard,Squamous
241,False,2.986648,62.424988,77.842548,4.084998,No,Standard,Smallcell


## 2. Feature Selection

Selección de las variables a utilizar en el modelo:
* `y (target)`: **event** y **time**
    * Para que estas variables puedan ser procesadas por el modelo, deben transformarse a otra estructura de datos (*numpy records array*). Esto se hace con *.to_records()*
* `x (explanatory)`: Variables relevantes para calcular el riesgo de un paciente.

### 2.1 Preprocessing Data

1. Revisar **NaN**: Eliminarlos del dataset
2. Transformar los datos categóricos de las variables Exploratory (X) a numéricos con **OneHotEncoder()**
    * La variable target (y) no necesita transformación de categóricos a numéricos ya que cuando se aplica el algoritmo de ML con .fit() este hace la transformación de forma automática.

In [63]:
df_patients.isna().sum()

event                    0
time                     0
age                      0
karnofsky_score          0
months_from_diagnosis    0
prior_therapy            0
treatment                0
celltype                 0
dtype: int64

In [64]:
y = df_patients[["event", "time"]].to_records(index=False)

In [65]:
X = df_patients.drop(["event", "time"], axis=1)
X

,age,karnofsky_score,months_from_diagnosis,prior_therapy,treatment,celltype
0,69.000000,60.000000,7.000000,No,Standard,Squamous
1,38.000000,60.000000,3.000000,No,Standard,Squamous
2,63.000000,60.000000,9.000000,Yes,Standard,Squamous
3,65.000000,70.000000,11.000000,Yes,Standard,Squamous
4,49.000000,20.000000,5.000000,No,Standard,Squamous
...,...,...,...,...,...,...
238,65.810046,64.640822,4.762009,No,Standard,Smallcell
239,36.495508,70.684273,21.551683,Yes,Test,Smallcell
240,65.029553,81.087920,4.852974,No,Standard,Squamous
241,62.424988,77.842548,4.084998,No,Standard,Smallcell


In [66]:
from sksurv.preprocessing import OneHotEncoder

In [67]:
encoder = OneHotEncoder()

In [68]:
X = encoder.fit_transform(X)

### 2.2 Train Test Split

In [69]:
from sklearn.model_selection import train_test_split

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [71]:
pd.DataFrame({
    "Dataset": ["X_train", "X_test", "y_train", "y_test"],
    "Registros": [len(X_train), len(X_test), len(y_train), len(y_test)]
})

,Dataset,Registros
0,X_train,170
1,X_test,73
2,y_train,170
3,y_test,73


## 3. The Cox PH Model

#### Importación del modelo y selección de hyperparameters

In [72]:
from sksurv.linear_model import CoxPHSurvivalAnalysis       # Se importa el algoritmo ML

In [73]:
model_cox = CoxPHSurvivalAnalysis()                         # Se instancia el modelo. De acá se sacan los hyperparametors (pasando el mouse por encima)

In [74]:
model_cox.get_params()                                      # Parámetros por defecto del modelo

{'alpha': 0, 'n_iter': 100, 'ties': 'breslow', 'tol': 1e-09, 'verbose': 0}

#### Grid Search CV - Selección de los mejores parámetros

In [75]:
from sklearn.model_selection import GridSearchCV

In [76]:
model_cv_cox = GridSearchCV(
    verbose=2,
    estimator=model_cox,
    param_grid={
        'ties': ['breslow', 'efron'],
        "n_iter" : [50, 100, 200, 300]
    }
)

In [77]:
model_cv_cox.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END ............................n_iter=50, ties=breslow; total time=   0.0s
[CV] END ............................n_iter=50, ties=breslow; total time=   0.0s
[CV] END ............................n_iter=50, ties=breslow; total time=   0.0s
[CV] END ............................n_iter=50, ties=breslow; total time=   0.0s
[CV] END ............................n_iter=50, ties=breslow; total time=   0.0s
[CV] END ..............................n_iter=50, ties=efron; total time=   0.0s
[CV] END ..............................n_iter=50, ties=efron; total time=   0.0s
[CV] END ..............................n_iter=50, ties=efron; total time=   0.0s
[CV] END ..............................n_iter=50, ties=efron; total time=   0.0s
[CV] END ..............................n_iter=50, ties=efron; total time=   0.0s
[CV] END ...........................n_iter=100, ties=breslow; total time=   0.0s
[CV] END ...........................n_iter=100, t

GridSearchCV(estimator=CoxPHSurvivalAnalysis(),
             param_grid={'n_iter': [50, 100, 200, 300],
                         'ties': ['breslow', 'efron']},
             verbose=2)

In [78]:
model_cv_cox.best_params_

{'n_iter': 50, 'ties': 'breslow'}

#### Score - Evaluación de las predicciones

In [79]:
model_cv_cox.score(X_test, y_test)

0.756129887342611

## 4. Decision Tree Model

#### Importación del modelo y selección de hyperparameters

In [80]:
from sksurv.tree import SurvivalTree

In [81]:
model_tree = SurvivalTree()

In [82]:
model_tree.get_params()

{'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_samples_leaf': 3,
 'min_samples_split': 6,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

#### Grid Search CV - Selección de los mejores parámetros

In [83]:
model_cv_tree = GridSearchCV(
    verbose = 2,
    estimator = model_tree,
    param_grid = {
        "splitter": ["best", "random"],
        "max_depth": [3, 5, 10, 15, 20],
        'max_features': [1, 2, 3, 4, 5, 6],
        'min_samples_leaf': [3, 5, 10]
    }
)

In [84]:
model_cv_tree.fit(X_train, y_train)

Fitting 5 folds for each of 180 candidates, totalling 900 fits
[CV] END max_depth=3, max_features=1, min_samples_leaf=3, splitter=best; total time=   0.0s
[CV] END max_depth=3, max_features=1, min_samples_leaf=3, splitter=best; total time=   0.0s
[CV] END max_depth=3, max_features=1, min_samples_leaf=3, splitter=best; total time=   0.0s
[CV] END max_depth=3, max_features=1, min_samples_leaf=3, splitter=best; total time=   0.0s
[CV] END max_depth=3, max_features=1, min_samples_leaf=3, splitter=best; total time=   0.0s
[CV] END max_depth=3, max_features=1, min_samples_leaf=3, splitter=random; total time=   0.0s
[CV] END max_depth=3, max_features=1, min_samples_leaf=3, splitter=random; total time=   0.0s
[CV] END max_depth=3, max_features=1, min_samples_leaf=3, splitter=random; total time=   0.0s
[CV] END max_depth=3, max_features=1, min_samples_leaf=3, splitter=random; total time=   0.0s
[CV] END max_depth=3, max_features=1, min_samples_leaf=3, splitter=random; total time=   0.0s
[CV] EN

[CV] END max_depth=10, max_features=6, min_samples_leaf=5, splitter=best; total time=   0.0s
[CV] END max_depth=10, max_features=6, min_samples_leaf=5, splitter=best; total time=   0.0s
[CV] END max_depth=10, max_features=6, min_samples_leaf=5, splitter=best; total time=   0.0s
[CV] END max_depth=10, max_features=6, min_samples_leaf=5, splitter=random; total time=   0.0s
[CV] END max_depth=10, max_features=6, min_samples_leaf=5, splitter=random; total time=   0.0s
[CV] END max_depth=10, max_features=6, min_samples_leaf=5, splitter=random; total time=   0.0s
[CV] END max_depth=10, max_features=6, min_samples_leaf=5, splitter=random; total time=   0.0s
[CV] END max_depth=10, max_features=6, min_samples_leaf=5, splitter=random; total time=   0.0s
[CV] END max_depth=10, max_features=6, min_samples_leaf=10, splitter=best; total time=   0.0s
[CV] END max_depth=10, max_features=6, min_samples_leaf=10, splitter=best; total time=   0.0s
[CV] END max_depth=10, max_features=6, min_samples_leaf=10

GridSearchCV(estimator=SurvivalTree(),
             param_grid={'max_depth': [3, 5, 10, 15, 20],
                         'max_features': [1, 2, 3, 4, 5, 6],
                         'min_samples_leaf': [3, 5, 10],
                         'splitter': ['best', 'random']},
             verbose=2)

In [85]:
model_cv_tree.best_params_

{'max_depth': 10, 'max_features': 4, 'min_samples_leaf': 5, 'splitter': 'best'}

In [86]:
model_cv_tree.predict(X_test)

array([ 40.95      ,  41.0952381 ,   0.        ,   8.5       ,
         0.        ,   8.5       ,   0.        ,  41.91666667,
        64.88809524, 120.84047619,  60.15      ,  28.67261905,
        28.67261905,   0.        , 120.84047619,   0.        ,
         8.5       , 124.35      ,  41.91666667,   0.        ,
        46.18571429,  46.18571429, 119.78333333,  73.33333333,
        15.46666667,   0.        ,  73.33333333,  22.        ,
        55.38333333,   0.        ,  41.91666667,   0.        ,
        46.18571429,  15.46666667, 119.78333333,   0.        ,
       120.84047619,  41.91666667,  41.91666667,  41.0952381 ,
        55.38333333, 124.35      ,  28.67261905,  41.91666667,
        41.91666667,  28.67261905,  41.91666667,  41.0952381 ,
         0.        ,  61.05      ,   0.        ,  46.18571429,
        55.38333333, 124.35      ,  73.33333333, 120.84047619,
        28.67261905,   0.        ,  41.0952381 ,  22.        ,
         0.        , 119.78333333,  41.91666667,  22.  

#### Score - Evaluación de las predicciones

In [87]:
model_cv_tree.score(X_test, y_test)

0.8750828363154407

## 5. Random Forest Model

#### Importación del modelo y selección de hyperparameters

In [88]:
from sksurv.ensemble import RandomSurvivalForest

In [89]:
model_rf = RandomSurvivalForest()

In [90]:
model_rf.get_params()

{'bootstrap': True,
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_samples_leaf': 3,
 'min_samples_split': 6,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

#### Grid Search CV - Selección de los mejores parámetros

In [91]:
model_cv_rf = GridSearchCV(
    verbose = 2,
    estimator = model_rf,
    param_grid = {
        "n_estimators": [50, 100, 200, 300],
        'min_samples_leaf': [2, 3, 5, 10]
    }
)

In [92]:
model_cv_rf.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END ................min_samples_leaf=2, n_estimators=50; total time=   0.0s
[CV] END ................min_samples_leaf=2, n_estimators=50; total time=   0.0s
[CV] END ................min_samples_leaf=2, n_estimators=50; total time=   0.0s
[CV] END ................min_samples_leaf=2, n_estimators=50; total time=   0.0s
[CV] END ................min_samples_leaf=2, n_estimators=50; total time=   0.0s
[CV] END ...............min_samples_leaf=2, n_estimators=100; total time=   0.0s


[CV] END ...............min_samples_leaf=2, n_estimators=100; total time=   0.0s
[CV] END ...............min_samples_leaf=2, n_estimators=100; total time=   0.0s
[CV] END ...............min_samples_leaf=2, n_estimators=100; total time=   0.0s
[CV] END ...............min_samples_leaf=2, n_estimators=100; total time=   0.0s
[CV] END ...............min_samples_leaf=2, n_estimators=200; total time=   0.1s
[CV] END ...............min_samples_leaf=2, n_estimators=200; total time=   0.1s
[CV] END ...............min_samples_leaf=2, n_estimators=200; total time=   0.2s
[CV] END ...............min_samples_leaf=2, n_estimators=200; total time=   0.2s
[CV] END ...............min_samples_leaf=2, n_estimators=200; total time=   0.2s
[CV] END ...............min_samples_leaf=2, n_estimators=300; total time=   0.2s
[CV] END ...............min_samples_leaf=2, n_estimators=300; total time=   0.2s
[CV] END ...............min_samples_leaf=2, n_estimators=300; total time=   0.2s
[CV] END ...............min_

GridSearchCV(estimator=RandomSurvivalForest(),
             param_grid={'min_samples_leaf': [2, 3, 5, 10],
                         'n_estimators': [50, 100, 200, 300]},
             verbose=2)

In [93]:
model_cv_rf.best_params_

{'min_samples_leaf': 2, 'n_estimators': 50}

#### Score - Evaluación de las predicciones

In [94]:
model_cv_rf.score(X_test, y_test)

0.8906560636182903

## 6. Support Vector Machine

#### Importación del modelo y selección de hyperparameters

In [95]:
from sksurv.svm import FastSurvivalSVM

In [96]:
model_svm = FastSurvivalSVM()

In [97]:
model_svm.get_params()

{'alpha': 1,
 'fit_intercept': False,
 'max_iter': 20,
 'optimizer': None,
 'random_state': None,
 'rank_ratio': 1.0,
 'timeit': False,
 'tol': None,
 'verbose': False}

#### Grid Search CV - Selección de los mejores parámetros

In [98]:
model_cv_svm = GridSearchCV(
    verbose = 2,
    estimator = model_svm,
    param_grid = {
        "alpha": [1, 2, 3],
        "max_iter": [5, 10, 20, 40]
    }
)

In [99]:
model_cv_svm.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ................................alpha=1, max_iter=5; total time=   0.0s
[CV] END ................................alpha=1, max_iter=5; total time=   0.0s
[CV] END ................................alpha=1, max_iter=5; total time=   0.0s
[CV] END ................................alpha=1, max_iter=5; total time=   0.0s
[CV] END ................................alpha=1, max_iter=5; total time=   0.0s
[CV] END ...............................alpha=1, max_iter=10; total time=   0.0s
[CV] END ...............................alpha=1, max_iter=10; total time=   0.0s
[CV] END ...............................alpha=1, max_iter=10; total time=   0.0s


c:\Users\guill\anaconda3\envs\ml\Lib\site-packages\sklearn\model_selection\_validation.py:686: ConvergenceWarning: Optimization did not converge: Warning: Maximum number of iterations has been exceeded.
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\guill\anaconda3\envs\ml\Lib\site-packages\sklearn\model_selection\_validation.py:686: ConvergenceWarning: Optimization did not converge: Warning: Maximum number of iterations has been exceeded.
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\guill\anaconda3\envs\ml\Lib\site-packages\sklearn\model_selection\_validation.py:686: ConvergenceWarning: Optimization did not converge: Warning: Maximum number of iterations has been exceeded.
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\guill\anaconda3\envs\ml\Lib\site-packages\sklearn\model_selection\_validation.py:686: ConvergenceWarning: Optimization did not converge: Warning: Maximum number of iterations has been exceeded.
  estimator.fit(X_train, y_train, **fit_para

[CV] END ...............................alpha=1, max_iter=10; total time=   0.0s
[CV] END ...............................alpha=1, max_iter=10; total time=   0.0s
[CV] END ...............................alpha=1, max_iter=20; total time=   0.0s
[CV] END ...............................alpha=1, max_iter=20; total time=   0.0s
[CV] END ...............................alpha=1, max_iter=20; total time=   0.0s


[CV] END ...............................alpha=1, max_iter=20; total time=   0.0s
[CV] END ...............................alpha=1, max_iter=20; total time=   0.0s
[CV] END ...............................alpha=1, max_iter=40; total time=   0.0s
[CV] END ...............................alpha=1, max_iter=40; total time=   0.0s
[CV] END ...............................alpha=1, max_iter=40; total time=   0.0s
[CV] END ...............................alpha=1, max_iter=40; total time=   0.0s
[CV] END ...............................alpha=1, max_iter=40; total time=   0.0s
[CV] END ................................alpha=2, max_iter=5; total time=   0.0s
[CV] END ................................alpha=2, max_iter=5; total time=   0.0s
[CV] END ................................alpha=2, max_iter=5; total time=   0.0s
[CV] END ................................alpha=2, max_iter=5; total time=   0.0s
[CV] END ................................alpha=2, max_iter=5; total time=   0.0s
[CV] END ...................

c:\Users\guill\anaconda3\envs\ml\Lib\site-packages\sklearn\model_selection\_validation.py:686: ConvergenceWarning: Optimization did not converge: Warning: Maximum number of iterations has been exceeded.
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\guill\anaconda3\envs\ml\Lib\site-packages\sklearn\model_selection\_validation.py:686: ConvergenceWarning: Optimization did not converge: Warning: Maximum number of iterations has been exceeded.
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\guill\anaconda3\envs\ml\Lib\site-packages\sklearn\model_selection\_validation.py:686: ConvergenceWarning: Optimization did not converge: Warning: Maximum number of iterations has been exceeded.
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\guill\anaconda3\envs\ml\Lib\site-packages\sklearn\model_selection\_validation.py:686: ConvergenceWarning: Optimization did not converge: Warning: Maximum number of iterations has been exceeded.
  estimator.fit(X_train, y_train, **fit_para

...

c:\Users\guill\anaconda3\envs\ml\Lib\site-packages\sklearn\model_selection\_validation.py:686: ConvergenceWarning: Optimization did not converge: Warning: Maximum number of iterations has been exceeded.
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\guill\anaconda3\envs\ml\Lib\site-packages\sklearn\model_selection\_validation.py:686: ConvergenceWarning: Optimization did not converge: Warning: Maximum number of iterations has been exceeded.
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\guill\anaconda3\envs\ml\Lib\site-packages\sklearn\model_selection\_validation.py:686: ConvergenceWarning: Optimization did not converge: Warning: Maximum number of iterations has been exceeded.
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\guill\anaconda3\envs\ml\Lib\site-packages\sklearn\model_selection\_validation.py:686: ConvergenceWarning: Optimization did not converge: Warning: Maximum number of iterations has been exceeded.
  estimator.fit(X_train, y_train, **fit_para

[CV] END ...............................alpha=3, max_iter=10; total time=   0.0s
[CV] END ...............................alpha=3, max_iter=10; total time=   0.0s
[CV] END ...............................alpha=3, max_iter=20; total time=   0.0s
[CV] END ...............................alpha=3, max_iter=20; total time=   0.0s
[CV] END ...............................alpha=3, max_iter=20; total time=   0.0s
[CV] END ...............................alpha=3, max_iter=20; total time=   0.0s


c:\Users\guill\anaconda3\envs\ml\Lib\site-packages\sklearn\model_selection\_validation.py:686: ConvergenceWarning: Optimization did not converge: Warning: Maximum number of iterations has been exceeded.
  estimator.fit(X_train, y_train, **fit_params)


[CV] END ...............................alpha=3, max_iter=20; total time=   0.0s
[CV] END ...............................alpha=3, max_iter=40; total time=   0.0s
[CV] END ...............................alpha=3, max_iter=40; total time=   0.0s
[CV] END ...............................alpha=3, max_iter=40; total time=   0.0s
[CV] END ...............................alpha=3, max_iter=40; total time=   0.0s
[CV] END ...............................alpha=3, max_iter=40; total time=   0.0s


GridSearchCV(estimator=FastSurvivalSVM(),
             param_grid={'alpha': [1, 2, 3], 'max_iter': [5, 10, 20, 40]},
             verbose=2)

#### Score - Evaluación de las predicciones

In [100]:
model_cv_svm.score(X_test, y_test)

0.7461895294897283

## 7. Resultado de los modelos

In [101]:
columnas = ["Modelo", "Score"]

In [102]:
df_resultados = pd.DataFrame({
    "Modelos": ["Cox PH", "Decision Tree", "Random Forest", "SVM"],
    "Score": [model_cv_cox.score(X_test,y_test), model_cv_tree.score(X_test,y_test), model_cv_rf.score(X_test,y_test), model_cv_svm.score(X_test,y_test)]
    }
    )
df_resultados.style.background_gradient()

,Modelos,Score
0,Cox PH,0.756130
1,Decision Tree,0.875083
2,Random Forest,0.890656
3,SVM,0.746190


## 8. Métricas de Evaluación

#### Indice de Concordancia (C-Index)  
El C-index en el análisis de supervivencia es una medida que nos dice qué tan bueno es un modelo en predecir quién vivirá más tiempo. 
* Un valor más alto (cerca de 1) es mejor, indica que el modelo es preciso. 
* Un valor bajo (cerca de 0.5) significa que el modelo no es mejor que adivinar al azar.
* El C-index te ayuda a saber si el modelo es bueno en ***ordenar a estos pacientes*** según cuánto tiempo viven. 


Cuando se habla de ***"ordenar a estos pacientes"*** significa que el modelo intenta poner a estos pacientes en una lista de manera que los que tienen una probabilidad más alta de vivir más tiempo estén al principio de la lista, y los que tienen una probabilidad más baja estén al final de la lista.

El C-index evalúa qué tan bien este ordenamiento coincide con la realidad.
* Un C-index alto significa que el modelo es bueno para poner a los pacientes en el orden correcto según su tiempo de supervivencia esperado. 
* Un C-index bajo significa que el modelo no es efectivo en esta tarea y no puede distinguir bien quién vivirá más tiempo y quién no.

----

### Bosquejo de la APP

Se creará una función la cual se alimentará con un diccionario.  
El diccionario contendrá los datos ingresados por el usuario/doctor en la app (age, karnoofsky_score, months_from_diagnosis, etc).  
Con esta información del diccionario, se calculará la probabilidad de muerte en un tiempo dado.

#### Cómo crear la función que tomará el diccionario y generará las predicciones

In [103]:
dic_input = {
    'age': 69.0,
    'karnofsky_score': 60.0,
    'months_from_diagnosis': 7.0,
    'prior_therapy': 'No',
    'treatment': 'Standard',
    'celltype': 'Squamous'
}

In [110]:
import sys
sys.path.append("..")
import data_utils

In [111]:
model_cv_rf.best_estimator_

RandomSurvivalForest(min_samples_leaf=2, n_estimators=50)

In [112]:
model = model_cv_rf.best_estimator_

In [113]:
data_utils.predict_survival_probability(dic_input, model, encoder)       # Se importa la función predict_survival_probability() del archivo data_utils.py

(47.53, 4.75)

#### La función Funciona!
Se puede decir que una persona con las características del dic_input tiene una probabilidad de morir de 47.53% en 4,75 meses.

### Exportación del Mejor Modelo

In [114]:
import pickle

In [115]:
# Vamos a definir el file "f" que contiene el modelo
f = open("../data/model.pkl", "wb")             # parametros "wb": Write Binary Code, es decir, lo va a escribir
pickle.dump(model, f)
f.close()

### Exportación del Encoder

In [116]:
encoder

OneHotEncoder()

In [118]:
f = open("../data/encoder.pkl", "wb")
pickle.dump(encoder, f)
f.close()